# Assignment 9

In [1]:
import numpy as np
import pandas as pd

### Part A

Consider a two period binomial model. Suppose after each period, the stock will either rise in value by ```r``` percent, or fall in value by ```f``` percent. Currently, the stock price is ```P```, and the risk free rate is ```rf```.

Write the function ```european_put```, which calculates the value of a one year European Put option with a strike price of ```K```. 

In [2]:
def european_put(P, K, r, f, rf):
    # Write your code here
    u = 1 + r
    d = 1 + f

    probability_fall = (np.sqrt(1 + rf) - d) / (u - d)

    S_ff = P * d * d

    # Calculate option values at terminal nodes
    V_dd = max(0, K - S_ff)

    # Calculate the option values at the earlier nodes
    V_d = np.sqrt(1/(1+rf)) * (1 - probability_fall) * (K - S_ff)

    # Calculate the option value at the initial node
    value = np.sqrt(1/(1+rf)) * (1 - probability_fall) * V_d

    return value  
            
# european_put(40, 40, 0.25, -0.2, 0.1) should return 2.616...
# european_put(40,40, 0.25, -0.2, 0.1)

Write the function ```american_put```, which calculates the value of a one year American Put option with a strike price of ```K```. 

In [3]:
def american_put(P, K, r, f, rf):
    # Write your code here
    u = 1 + r
    d = 1 + f

    # Calculate the probabilities of up and down movements
    pu = (np.sqrt(1 + rf) - d) / (u - d)
    pd = 1 - pu
    
    S_d = P * d

    V_0 = (pd * (K - S_d)) / np.sqrt(1 + rf)

    return V_0

# american_put(40,40, 0.25, -0.2, 0.1) should return 3.410...

#american_put(40, 40, 0.25, -0.2, 0.1)

### Part B

In addition to simple calls and puts, a wide variety of exotic options are traded in over-the-counter markets. In most cases, there are no analytic formulas available to value these options, and so numerical techniques must be used. One example of these techniques are Monte Carlo methods. These are particularly useful in certain contexts, such as when option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. This question will explore some basic examples, and it will require relatively large number of paths. Assume that the stock price will be needed at the end of each of the next 12 months, in addition to S0 today, and start by simulating `N` such stock price paths. In particular:

•	You have been provided the function ```gen_random_vars```, which generates random variables that follow a standard normal distribution (Note that you should never actually call ```gen_random_vars``` in any of your functions, this is simply meant to help give you inputs to help test your code). This outputs a table with `N` rows and 12 columns. Denote the value of the random variable on path i for month j by zi,j where i = 1, . . . ,`N` and j = 1, . . . ,12.

Write the function ```create_paths```, which takes as an argument a DataFrame ```random_vars```, the output of the function ```gen_random_vars```, and ```S0```, the initial stock price.

This function will generate a DataFrame with `N` stock price paths $S_{i,j}$ where $i = 1, . . . ,N$ and $j = 0, . . . ,12$. In each case, $S_{i,0} =$ ```S_0```. For the remaining entries in this table, define $∆t = 1/12$ and set

$$ S_{i,j} = S_{i, j-1} \exp [(r - \sigma^2 / 2) \Delta t + \sigma \sqrt{\Delta t} z_{i,j}]$$

 
for i = 1, . . . ,`N` and j = 1, . . . ,12. (If you are wondering why we are using the risk-free rate r here instead of the expected rate of return on the underlying stock, it is because we are using risk-neutral valuation.) 

Note that throughout the rest of this question, $S_T$ on each simulated path i is defined to be equal to $S_{i,12}$.



In [4]:
def gen_random_vars(SEED, N):
    rng = np.random.default_rng(SEED)

    zArr = np.array([rng.normal(size=N)])
    for i in range(11):
        z = rng.normal(size=N)
        zArr = np.r_[zArr,[z]]
    

    randomVariables = pd.DataFrame(zArr.transpose())
    return randomVariables



In [5]:
def create_paths(random_vars, S0, rf, sigma):
    # Do not rename the columns or rows of the outputted dataframe. 
    # The outputted DataFrame should have N rows and 13 columns.
    
    N = len(random_vars)
    dt = 1/12  # time step

    # Create a DataFrame to store stock price paths
    columns = [i for i in range(13)]
    paths_df = pd.DataFrame(index=range(N), columns=columns)

    # Set initial stock prices
    paths_df[0] = S0

    # Generate stock price paths
    for j in range(1, 13):
        # Calculate stock prices using the given formula
        paths_df[j] = paths_df.iloc[:, j-1] * np.exp((rf - (sigma ** 2) / 2) * dt + sigma * np.sqrt(dt) * random_vars.iloc[:, j-1])

    return paths_df

# If we let sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35),
# The last column of the first row of sample_paths should equal 71.56...
#create_paths(r_vars, 72.25, 0.04 , 0.35)

•	Write the function ```payoff_table```, which returns another DataFrame with `N` rows and 6 columns. The columns will be used to store discounted payoffs at expiration for various different options, as indicated below:

–	European call option, with payoff $max(S_T − K,0)$.

–	European put option, with payoff $max(K - S_T ,0)$.

–	Average price put option, with payoff $max(K − S^-,0)$, where
$S¯ = S_0 + S_1 + · · · + S_{12}$ .

–	Floating lookback call option, with payoff $S_T - S_{min}$, where $S_{min}$ on each path is the minimum of the stock prices $S_0, S_1, . . . , S_{12}$ on that path.

–	Up-and-out call option with barrier ```B```, with payoff $max(S_T - K,0)$ as long as none of the observed monthly stock prices on the given path are greater than or equal to ```B```. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.

–	Up-and-in call option with barrier ```B``` , with payoff $max(S_T - K,0)$ as long as at least one of the observed monthly stock prices on the given path is greater than or equal to ```B```. If this condition is not satisfied, the option payoff is zero.

Note that “discounted payoffs” implies that the entries in this table should have the payoff functions multiplied by $e^{−rT}$ , where T = 1.

This function will also take in a dataframe ```paths```, which is the output from the previous function ```create_paths```.

In [17]:
def payoff_table(paths, K, T, rf, B):
        
    N = len(paths)

    # Calculate average price at expiration
    average_price = paths.iloc[:, 0:].mean(axis=1)

    # Calculate minimum stock price at expiration for each path
    S_min = paths.iloc[:, 0:].min(axis=1)

    # Calculate discounted payoffs for each option
    european_call = np.maximum(paths[12] - K, 0) * np.exp(-rf * T)
    european_put = np.maximum(K - paths[12], 0) * np.exp(-rf * T)
    avg_price_put = np.maximum(K - average_price, 0) * np.exp(-rf * T)
    floating_lookback = np.maximum(paths[12] - S_min, 0) * np.exp(-rf * T)

    # Up-and-out call option
    up_and_out_call = np.where((paths.iloc[:, 0:] >= B).any(axis=1), 0, european_call)

    # Up-and-in call option
    up_and_in_call = np.where((paths.iloc[:, 0:] >= B).any(axis=1), european_call, 0)

    # Create DataFrame with payoffs
    answer = pd.DataFrame({
        "European Call": european_call,
        "European Put": european_put,
        "Average Price Put": avg_price_put,
        "Floating Lookback": floating_lookback,
        "Up-and-out": up_and_out_call,
        "Up-and-in": up_and_in_call
    })

    return answer
            
# If we let sample_payoffs = payoff_table(sample_paths, 70, 1, 0.04, 90)
# The payoff of the first European Call should be 1.500...

sample_paths = create_paths(r_vars, 72.25, 0.04 , 0.35)
#payoff_table(create_paths(r_vars, 72.25, 0.04 , 0.35), 70, 1, 0.04, 90)


NameError: name 'r_vars' is not defined

Write the function ```statistics```, which takes as input a DataFrame ```payoffs```, formatted as one produced by ```payoff_table```, and displays the following statistics for each of the options above (below N is the number of simulated paths):

•	Mean discounted payoff µ. This is the estimated option premium.

•	Standard error of option premium ϕ. This is the standard deviation of the option payoff divided by √N .

•	Lower bound for 95% confidence interval for option premium, which is µ - 1.96ϕ.

•	Upper bound for 95% confidence interval for option premium, which is µ + 1.96ϕ.

The size of the returned DataFrame should have 6 rows and 4 columns. 

In [16]:
def statistics(payoffs):
    
    # Note that the index rows have been renamed here. Ensure that your rows match the order 
    # that the indices have been renamed as.

    N = len(payoffs)
    
    # Define the order of indices for renaming
    index_order = ["European Call", "European Put", "Average Price Put", "Floating Lookback", "Up-and-out", "Up-and-in"]

    # Create an output DataFrame with the specified index order
    output = pd.DataFrame(index=index_order, columns=["Mean", "Standard Error", "Lower Bound", "Upper Bound"])

    for i, option in enumerate(index_order):
        payoff_values = payoffs.iloc[:, i]

        # Mean discounted payoff (estimated option premium)
        mean_payoff = np.mean(payoff_values)

        # Standard error of option premium
        std_error = np.std(payoff_values) / np.sqrt(N)

        # Confidence interval bounds
        lower_bound = mean_payoff - 1.96 * std_error
        upper_bound = mean_payoff + 1.96 * std_error

        # Fill in the output DataFrame
        output.loc[option] = [mean_payoff, std_error, lower_bound, upper_bound]

    # output = pd.DataFrame(data = {
    #     "Mean" : [],
    #     "Standard Error" : [],
    #     "Lower Bound" : [],
    #     "Upper Bound" : []
    # }).rename(index={0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}, errors="raise")
    
    return output

# If we call statistics(sample_payoffs), the mean of the European Puts should be 7.370...
sample_payoffs = payoff_table(create_paths(r_vars, 72.25, 0.04 , 0.35), 70, 1, 0.04, 90)
statistics(sample_payoffs)

NameError: name 'r_vars' is not defined

One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, there has been a lot of research into variance reduction methods, in an effort to improve the rate of convergence. 

One of the simplest such approaches is called antithetic variates. This relies on the fact that if Z is a standard normal random variable, then so is -Z. To see how this can be used, generate another `N` stock price paths, but this time using $-z_{i,j}$ instead of $z_{i,j}$ throughout.

In order to accomplish this, write the function ```create_negative_paths```. This function has the same arguments as ```create_paths```, but creates the stock prices in the manner described above.

In [8]:
def create_negative_paths(random_vars, S0, rf, sigma):
    # Write your code here
    N = len(R)
    dt = 1/12 

    # DataFrame to store stock price paths
    columns = [i for i in range(13)]
    paths_df = pd.DataFrame(index=range(N), columns=columns)

    # set initial stock prices
    paths_df[0] = S0

    for j in range(1, 13):
        # use -z instead of z
        paths_df.iloc[:, j] = paths_df.iloc[:, j-1] * np.exp((rf - (sigma ** 2) / 2) * dt - sigma * np.sqrt(dt) * random_vars.iloc[:, j-1])

    return paths_df

#neg_paths = create_negative_paths(r_vars, 72.25, 0.04 , 0.35)
#create_negative_paths(r_vars, 72.25, 0.04 , 0.35)


Write the function ```antithetic_statistics```.

In this function, you will use ```payoff_table``` to create two tables, one using paths generated from ```create_paths```, and one generated from ```create_negative_paths```.

Next, create a third table which contains the average payoff between the two tables.

Finally, call ```statistics``` on this third table. Return the table produced by ```statistics```.

If you compare this table with the output of `statistics` (as above, without creating two tables), you should notice that Mean, Lower Bound, and Upper Bound columns should have relatively similar values, while the Standard Error column has decreased significantly.

In [9]:
def antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    # Write your code here

    t1 = payoff_table(create_paths(random_vars, S0, rf, sigma), K, T, rf, B)
    t2 = payoff_table(create_negative_paths(random_vars, S0, rf, sigma), K, T, rf, B)
    t3 = (t1 + t2)/2

    return statistics(t3)

#antithetic_statistics(r_vars, 72.25, 0.04, 0.35, 70, 1, 90)

In [10]:
def correct_create_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*randomVariables[j][i])
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_payoff_table(paths, K, T, rf, B):
    N = len(paths)
    #discounts a payment by a year per the question specifications
    def discount(pmt):
        return pmt*np.exp(-rf * T)

    #functions that calculate payoff for each option given a price path
    def EuropeanCall(path):
        return discount(max(path[12] - K, 0))

    def EuropeanPut(path):
        return discount(max(K-path[12], 0))
    
    def AveragePricePut(path):
        Sbar = path.mean()
        return discount(max(K - Sbar, 0))
    
    def FloatingLookback(path):
        Smin = path.min()
        return discount(path[12] - Smin)
    
    def upAndOut(path, B):
        for i in range(13):
            if(path[i] >= B):
                return 0
        return discount(max(path[12] - K, 0))
    
    def upAndIn(path, B):
        atLeastOne = False
        for i in range(13):
            if(path[i] >= B):
                atLeastOne = True
                break
        if atLeastOne:
            St = path[12]
            return discount(max(St - K, 0))
        else:
            return 0
    
    payoffsArr = np.zeros((N, 6))
    
    for i in range(N):
        row = paths.iloc[i]
        payoffsArr[i][0] = EuropeanCall(row)
        payoffsArr[i][1] = EuropeanPut(row)
        payoffsArr[i][2] = AveragePricePut(row)
        payoffsArr[i][3] = FloatingLookback(row)
        payoffsArr[i][4] = upAndOut(row, 90)
        payoffsArr[i][5] = upAndIn(row, 90)
        
    payoffs = pd.DataFrame(payoffsArr)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    payoffs = payoffs.rename(columns=renameDict, errors="raise")
    return payoffs

def correct_statistics(payoffs):
    N = len(payoffs)
    #Statistics
    statLst = []
    for column in payoffs:
        row = []
        info = payoffs[column]
        mean = info.mean()
        sd = info.std()
        se = sd/(N**0.5)
        row.append(mean)
        row.append(se)
        row.append(mean-1.96*se)
        row.append(mean+1.96*se)
        statLst.append(row)

    stats = pd.DataFrame(statLst)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    stats = stats.rename(index=renameDict, errors="raise")
    renameDict2 = {0: "Mean", 1: "Standard Error", 2: "Lower Bound", 3: "Upper Bound"}
    stats = stats.rename(columns=renameDict2, errors="raise")
    return stats



def correct_create_negative_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*(-randomVariables[j][i]))
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    paths1 = correct_create_paths(random_vars, S0, rf, sigma)
    paths2 = correct_create_negative_paths(random_vars, S0, rf, sigma)
    payoffs1 = correct_payoff_table(paths1, K, T, rf, B)
    payoffs2 = correct_payoff_table(paths2, K, T, rf, B)
    
    avg_payoffs = (payoffs1 + payoffs2)/2
    return correct_statistics(avg_payoffs)

test_r_vars = gen_random_vars(700, 20)

test_paths = correct_create_paths(test_r_vars, 72.25, 0.04 , 0.35)

test_payoff_table = correct_payoff_table(test_paths, 70, 1, 0.04, 90)

In [11]:
payoff_table(test_paths, 70, 1, 0.04, 90)

European Call  European Put  Average Price Put  Floating Lookback  \
0       23.779881      0.000000           0.000000          30.977618   
1        0.000000      9.457377           4.597758           2.323336   
2        0.000000     14.515353           0.000000           0.000000   
3       13.118333      0.000000           0.000000          17.477566   
4        0.000000     16.711348           5.605504           1.638815   
5        7.359926      0.000000           0.000000          13.543289   
6       20.282395      0.000000           0.000000          24.369938   
7       55.433441      0.000000           0.000000          56.318530   
8       14.574302      0.000000           0.000000          15.792084   
9        0.000000     19.256275           5.972241           0.000000   
10      25.084811      0.000000           0.000000          24.278997   
11      68.592737      0.000000           0.000000          71.636659   
12       0.000000     28.582690          21.025465           2.038402   
13       0.000000     34.810008          10.799256           4.217981   
14       0.000000      2.360981           0.000000           0.338367   
15       6.180395      0.000000           0.000000           4.109110   
16       0.000000     30.975041          20.408709           3.577003   
17       0.000000      7.165322           4.657227           8.429508   
18       0.000000      4.121468           1.356282           7.793792   
19       0.000000     34.490468          18.651254           1.434599   

    Up-and-out  Up-and-in  
0     0.000000  23.779881  
1     0.000000   0.000000  
2     0.000000   0.000000  
3    13.118333   0.000000  
4     0.000000   0.000000  
5     7.359926   0.000000  
6     0.000000  20.282395  
7     0.000000  55.433441  
8     0.000000  14.574302  
9     0.000000   0.000000  
10    0.000000  25.084811  
11    0.000000  68.592737  
12    0.000000   0.000000  
13    0.000000   0.000000  
14    0.000000   0.000000  
15    0.000000   6.180395  
16    0.000000   0.000000  
17    0.000000   0.000000  
18    0.000000   0.000000  
19    0.000000   0.000000

In [12]:
antithetic_statistics(test_r_vars, 72.25, 0.04, 0.35,  70, 1, 90)

NameError: name 'R' is not defined

In [ ]:
create_negative_paths

In [ ]:
correct_create_negative_paths(test_r_vars, 72.25, 0.04 , 0.35)

In [ ]:
create_negative_paths(test_r_vars, 72.25, 0.04 , 0.35)

In [13]:
def correct_create_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*randomVariables[j][i])
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_payoff_table(paths, K, T, rf, B):
    N = len(paths)
    #discounts a payment by a year per the question specifications
    def discount(pmt):
        return pmt*np.exp(-rf * T)

    #functions that calculate payoff for each option given a price path
    def EuropeanCall(path):
        return discount(max(path[12] - K, 0))

    def EuropeanPut(path):
        return discount(max(K-path[12], 0))
    
    def AveragePricePut(path):
        Sbar = path.mean()
        return discount(max(K - Sbar, 0))
    
    def FloatingLookback(path):
        Smin = path.min()
        return discount(path[12] - Smin)
    
    def upAndOut(path, B):
        for i in range(13):
            if(path[i] >= B):
                return 0
        return discount(max(path[12] - K, 0))
    
    def upAndIn(path, B):
        atLeastOne = False
        for i in range(13):
            if(path[i] >= B):
                atLeastOne = True
                break
        if atLeastOne:
            St = path[12]
            return discount(max(St - K, 0))
        else:
            return 0
    
    payoffsArr = np.zeros((N, 6))
    
    for i in range(N):
        row = paths.iloc[i]
        payoffsArr[i][0] = EuropeanCall(row)
        payoffsArr[i][1] = EuropeanPut(row)
        payoffsArr[i][2] = AveragePricePut(row)
        payoffsArr[i][3] = FloatingLookback(row)
        payoffsArr[i][4] = upAndOut(row, 90)
        payoffsArr[i][5] = upAndIn(row, 90)
        
    payoffs = pd.DataFrame(payoffsArr)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    payoffs = payoffs.rename(columns=renameDict, errors="raise")
    return payoffs

def correct_statistics(payoffs):
    N = len(payoffs)
    #Statistics
    statLst = []
    for column in payoffs:
        row = []
        info = payoffs[column]
        mean = info.mean()
        sd = info.std()
        se = sd/(N**0.5)
        row.append(mean)
        row.append(se)
        row.append(mean-1.96*se)
        row.append(mean+1.96*se)
        statLst.append(row)

    stats = pd.DataFrame(statLst)
    renameDict = {0: "European Call", 1: "European Put", 2: "Average Price Put", 3: "Floating Lookback",4: "Up-and-out",5: "Up-and-in"}
    stats = stats.rename(index=renameDict, errors="raise")
    renameDict2 = {0: "Mean", 1: "Standard Error", 2: "Lower Bound", 3: "Upper Bound"}
    stats = stats.rename(columns=renameDict2, errors="raise")
    return stats



def correct_create_negative_paths(random_vars, S0, rf, sigma):
    randomVariables = random_vars
    N = len(random_vars)
    deltaT = 1/12

    pathArr = []

    for i in range(N):
        row = [S0]
        for j in range(12):
            newPrice = row[-1] * np.exp((rf - (sigma**2)/2)*deltaT + sigma*((deltaT**0.5))*(-randomVariables[j][i]))
            row.append(newPrice)
        pathArr.append(row)
    
    paths = pd.DataFrame(pathArr)
    return paths

def correct_antithetic_statistics(random_vars, S0, rf, sigma, K, T, B):
    paths1 = correct_create_paths(random_vars, S0, rf, sigma)
    paths2 = correct_create_negative_paths(random_vars, S0, rf, sigma)
    payoffs1 = correct_payoff_table(paths1, K, T, rf, B)
    payoffs2 = correct_payoff_table(paths2, K, T, rf, B)
    
    avg_payoffs = (payoffs1 + payoffs2)/2
    return correct_statistics(avg_payoffs)

test_r_vars = gen_random_vars(700, 20)

test_paths = correct_create_paths(test_r_vars, 72.25, 0.04 , 0.35)

test_payoff_table = correct_payoff_table(test_paths, 70, 1, 0.04, 90)

In [14]:
correct_statistics(test_payoff_table)

Mean  Standard Error  Lower Bound  Upper Bound
European Call      11.720311        4.326628     3.240121    20.200501
European Put       10.122317        2.880351     4.476829    15.767804
Average Price Put   4.653685        1.623593     1.471442     7.835928
Floating Lookback  14.514780        4.342248     6.003973    23.025586
Up-and-out          1.023913        0.735010    -0.416707     2.464533
Up-and-in          10.696398        4.396854     2.078563    19.314233

In [15]:
statistics(test_payoff_table)

Mean Standard Error Lower Bound Upper Bound
European Call      11.720311       4.217075    3.454844   19.985778
European Put       10.122317       2.807419    4.619776   15.624857
Average Price Put   4.653685       1.582483    1.552018    7.755352
Floating Lookback   14.51478         4.2323    6.219472   22.810088
Up-and-out          1.023913       0.716399    -0.38023    2.428056
Up-and-in          10.696398       4.285524    2.296772   19.096024